## AUTOARIMA

In [30]:
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima
from pmdarima.model_selection import train_test_split
import pandas as pd
from src.drawing import show_forecast_test_plot, show_time_series_only, show_acf_pacf
from src.statistic import check_stationarity, make_diff_n_degree
from statsmodels.tsa.arima.model import ARIMA

In [56]:
ts = pd.read_csv('train.csv')
time = ts.Date
target = ts.target_value
TRAIN_SIZE = len(target) - 7
train, test = train_test_split(target, train_size=TRAIN_SIZE)

In [57]:
model = auto_arima(train,m=30, trace=True)
forecasts = model.predict(n_periods=7)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[30] intercept   : AIC=3793.175, Time=1.55 sec
 ARIMA(0,1,0)(0,0,0)[30] intercept   : AIC=4021.795, Time=0.01 sec
 ARIMA(1,1,0)(1,0,0)[30] intercept   : AIC=3912.194, Time=0.30 sec
 ARIMA(0,1,1)(0,0,1)[30] intercept   : AIC=3786.114, Time=0.66 sec
 ARIMA(0,1,0)(0,0,0)[30]             : AIC=4024.182, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[30] intercept   : AIC=3784.280, Time=0.09 sec
 ARIMA(0,1,1)(1,0,0)[30] intercept   : AIC=3786.116, Time=0.45 sec
 ARIMA(0,1,1)(1,0,1)[30] intercept   : AIC=3788.112, Time=1.27 sec
 ARIMA(1,1,1)(0,0,0)[30] intercept   : AIC=3786.163, Time=0.07 sec
 ARIMA(0,1,2)(0,0,0)[30] intercept   : AIC=3786.169, Time=0.08 sec
 ARIMA(1,1,0)(0,0,0)[30] intercept   : AIC=3910.199, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[30] intercept   : AIC=3787.723, Time=0.13 sec
 ARIMA(0,1,1)(0,0,0)[30]             : AIC=3784.483, Time=0.03 sec

Best model:  ARIMA(0,1,1)(0,0,0)[30] intercept
Total fit time: 4.679 seconds


In [58]:
show_time_series_only(time, target)

In [59]:
show_forecast_test_plot(time,train, forecasts, test)

Видно что AUTO-ARIMA как то справляется с трендом в принципе если ее усилить чем-то то можно будет уже очень даже хорошо предсказывать, но уже и так есть огромный плюс то что она показывает тренд правильно

## ARIMA FROM SCRATCH

In [8]:
target = ts.target_value
check_stationarity(target)

ADF Statistic: -1.804660
p-value: 0.378151
Critical Values:
	1%: -3.449
	5%: -2.870
	10%: -2.571
The TS is NOT stationary


In [22]:
%matplotlib notebook

target = ts.target_value
show_time_series_only(ts.Date, target)
check_stationarity(target)
target = make_diff_n_degree(target, 1)
check_stationarity(target)
show_time_series_only(ts.Date, target)
show_acf_pacf(target, 181) # ACF 2 , PACF 68

ADF Statistic: -1.804660
p-value: 0.378151
Critical Values:
	1%: -3.449
	5%: -2.870
	10%: -2.571
The TS is NOT stationary
ADF Statistic: -5.054608
p-value: 0.000017
Critical Values:
	1%: -3.449
	5%: -2.870
	10%: -2.571
The TS is stationary


<IPython.core.display.Javascript object>

In [24]:
print('total 1 times diff req')
print('so d is 1')

total 1 times diff req
so d is 1


In [25]:
print('p = 1')

p = 1


In [26]:
print('q=2')

q=2


In [47]:
arima_model = ARIMA(ts.target_value[:359], order = (2, 1, 2))
model_fit = arima_model.fit()
forecast = model_fit.forecast(steps=7)
print(model.summary())

                               SARIMAX Results                                
Dep. Variable:           target_value   No. Observations:                  359
Model:                 ARIMA(1, 1, 2)   Log Likelihood               -1889.777
Date:                Wed, 24 May 2023   AIC                           3787.553
Time:                        14:36:31   BIC                           3803.076
Sample:                             0   HQIC                          3793.727
                                - 359                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.6573      0.572      1.150      0.250      -0.463       1.778
ma.L1          0.3438      0.592      0.581      0.561      -0.816       1.504
ma.L2         -0.5956      0.574     -1.038      0.2

In [48]:
show_forecast_test_plot(ts.Date, ts.target_value[:359], forecast, ts.target_value[359:])

In [55]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

data = pd.read_csv('train.csv')
train =  data.target_value[:-7]
test = data.target_value[-7:]
time = data.Date

model = SARIMAX(train, order=(0, 1, 1), seasonal_order=(0, 0, 0, 0))
model_fit = model.fit()
forecast = model_fit.forecast(steps=7)

show_forecast_test_plot(time,train,forecast, test)